# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e compramos e vendemos commodities:
- Soja, Milho, Trigo, Petróleo, etc.

Precisamos pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do nosso preço ideal de compra. Se tiver, precisamos marcar como uma ação de compra para a equipe de operações.

Base de Dados: https://drive.google.com/drive/folders/1O9aE_Hen5smf_a6TsbVF6DM0ms4Ykj6s?usp=share_link

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

Obtendo executavel do navegador chrome

In [86]:
import os
import re
# Retorna o caminho para o executável mais recente do Chrome x32 ou X86
# busca google dentro da pasta programfiles
results_32 = [x for x in os.listdir(os.environ['PROGRAMFILES']) if re.search(r'(google)', x, re.IGNORECASE)]
# busca google dentro da pasta programfiles (x86)
results_64 = [x for x in os.listdir(os.environ['PROGRAMFILES(X86)']) if re.search(r'(google)', x, re.IGNORECASE)]
# Retorna o navegador da base de dados.
if results_32:
    # monta o caminho do executavel
    chrome_path = os.path.join(os.environ['PROGRAMFILES'], results_32[0], 'Chrome', 'Application')
    # obtem apenas executavel do caminho montado
    files = [x for x in os.listdir(chrome_path) if re.search(r'.exe$', x)]
    # Retorna o primeiro arquivo na lista de arquivos.
    if len(files) == 1:
        file = files[0]
    else:
        # caso haja mais de um executável
        file = [x for x in files if re.search(r'chrome.exe', x)][0]
    # caminho do executavel x32
    browser_path = os.path.join(chrome_path,file)
elif results_64:
    # obtem apenas executavel do caminho montado
    chrome_path = os.path.join(os.environ['PROGRAMFILES(X86)'], results_64[0], 'Chrome', 'Application')
    # verificar se existe o executável
    files = [x for x in os.listdir(chrome_path) if re.search(r'.exe$', x)]
    # Retorna o primeiro arquivo na lista de arquivos.
    if len(files) == 1:
        file = files[0]
    else:
        # caso haja mais de um executável
        file = [x for x in files if re.search(r'chrome.exe', x)][0]
    # caminho do executavel x64
    browser_path = os.path.join(chrome_path, file)
print(browser_path)

C:\Program Files\Google\Chrome\Application\chrome.exe


Abrindo navegador

In [87]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
# instancia serviço do chrome e baixa o webdriver atualizado
service = Service(ChromeDriverManager().install())
# opções do navegador
chrome_options = Options()
chrome_options.binary_location = browser_path
navegador = webdriver.Chrome(options=chrome_options)
navegador.get("https://google.com")



Importar base de dados

In [88]:
import pandas as pd

tabela = pd.read_excel("commodities.xlsx")

tabela.head()

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN


Obter cotação dos commodities e atualizando na tabel

In [89]:
from selenium.webdriver.common.by import By
from unidecode import unidecode
for linha in tabela.index:
    produto = tabela.loc[linha, "Produto"]
    produto =  unidecode(produto)
    navegador.get(f"https://melhorcambio.com/{produto}-hoje")
    # utilizar find_elements para evitar o erro do find_element que quebra a aplicação, quando não encontra o elemento pois o selenium não trata isso
    # o find_elements quando não encontra o elemento ele retorna uma lista vazia
    elementos = navegador.find_elements(By.ID, "comercial")
    # 
    if elementos:
        # obtem o valor da cotação
        cotacao = elementos[0].get_attribute("value")
        cotacao = cotacao.replace(".", "").replace(",", ".")
        cotacao = float(cotacao)
        print(cotacao)
    else:
        # imprima a lista vazia
        print(elementos)

    tabela.loc[linha, "Preço Atual"] = cotacao

    
navegador.quit()

tabela["Comprar"] = tabela["Preço Atual"] < tabela["Preço Ideal"]
display(tabela)

57.43
133.82
267.75
379.22
390.52
148.92
1042.37
315.32
9.64


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,57.43,True
1,Soja,163.59,133.82,True
2,Boi,282.20,267.75,True
3,Petróleo,424.37,379.22,True
4,Algodão,497.76,390.52,True
5,Açúcar,136.23,148.92,False
6,Café,1092.87,1042.37,True
7,Ouro,321.77,315.32,True
8,Tilápia,9.05,9.64,False


Exportar a base de dados atualizada

In [90]:
tabela.to_excel("commodities_atualizado.xlsx", index=False)